In [ ]:
"""
Get articles/texts, and filter only sentences with white-listed vocab OR named entities.

Contents:
-load packages
-create helper functions
-data intialization from csv file of words &
 extend vocab set
-build data structure to house info moving forward
-write to .json formats
-read from existing .json formats
-get white-listed of words and inflections
-scrape journale en francais facile
"""

In [187]:
import bs4, requests, sys, codecs, urllib.request, re
from bs4 import SoupStrainer
from bs4.element import Comment
import random
import string
import json
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import mlconjug3
import sklearn
import goslate
from nltk.tokenize import word_tokenize, sent_tokenize
import pprint
pp = pprint.PrettyPrinter()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 0)
import mitosheet
import translators as ts

In [188]:
def add_noun(add_word):

    if add_word[len(add_word)-1] != 's' and add_word[len(add_word)-1] != 'x':

        out_word = add_word + 's'

        return out_word
    else:
        return add_word

In [189]:
##METHODS

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = bs4.BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)



user_agent = 'Mozilla/5.0 (Windows NT 6.3; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
headers={'User-Agent':user_agent,}


In [190]:
def get_known(unknown_list):
    
    count_got = 0
    known_list = []
    count_checked = 0
    for word in unknown_list:
        count_checked += 1
        decision = str(input(word+"\nKnown =k"))
        if decision =='k':
            known_list.append(word)
            count_got +=1
            print("got "+str(count_got))
        elif decision=='q':
            break
        print(str(len(unknown_list)-count_checked)+' remaining')
    return known_list

In [191]:
def review_out(out_dict,prev_learned,eb_unk_in,eb_unk_m_in,examples,counters):
    """
    out_dict = out
    prev_learned = eb_learned
    eb_unk_in =  eb_unk
    eb_unk_m_in = eb_unk_morph
    examples = eb_unk_examples
    
    decisions:
    k = word known, remove from eb_unk and eb_unk_morph
        and add to eb_learned
        
    m = remove morph from eb_unk and eb_unk_morph 
    
    a = add example to eb_unk_examples
    q = quit for day
    """
    print(str(len(out_dict.keys()))+" total")
    counter = 0
    
    for word in out_dict.keys():
        counter += 1
        print(str(counter)+" out of "+str(len(out_dict.keys())))
        print("Lemma: "+word)
        print('')
        print("Morph: "+str(out_dict[word]['morph']))
        
        

        if len(examples[word])>0:
            print("Previous examples: ")
            for sent in examples[word]:
                print(" - "+sent)

        print("New examples: ")

        i = 0
        decision = ''
        while (i < len(out_dict[word]['sents'])) and (decision!='s')\
        and (decision!='q'):

            sent = out_dict[word]['sents'][i]
            if (sent not in examples[word]) and (sent not in counters[word]):
                print(sent)
                print('')
                decision = str(input("options : k,m,a,q,s,c"))
                if decision =='k':
                    try:
                        examples[word].append(sent) #by default adds examples
                        prev_learned[word] = eb_unk_in[word].copy()
                        del eb_unk_in[word] #remove lemma

                        i = 0
                        while i < len(eb_unk_m_in):

                            if eb_unk_m_in[list(eb_unk_m_in)[i]] == word:
                               del eb_unk_m_in[list(eb_unk_m_in)[i]]
                            else:
                                i +=1
                    except:
                        pass

                if decision =='m':
                    morph = str(input("which morph:"))
                    try:
                        del eb_unk_m_in[morph]
                        eb_unk_in[word].remove(morph) 
                    except:
                        pass
                if decision =='a':
                    try:
                        examples[word].append(sent)
                    except:
                        pass
                if decision=='c':
                    counters[word].append(sent)
                if decision =='q':
                    break
            i += 1
    

In [192]:
def how_to_add(word):
    """
    p = plain, add without conjurgating or changing
    c = conjugate as normal verb
    x = noun that adds x when plural
    s = noun that add s when plural
    iv = inner verb. There is a verb in the expression that should be 
        conjurgated
    """
    extras_morphs = []
    extras_morphs.append(word)
    
    
    gs = goslate.Goslate()

    default_conjugator = mlconjug3.Conjugator(language='fr')

    decision = str(input(word+"\nc\nx\ns\niv"))
    
    if decision == 'c':

        try:
            temp = []
            test_verb = default_conjugator.conjugate(word)
            all_conjugated_forms = test_verb.iterate()
            for item in all_conjugated_forms:
                if item not in temp:
                    temp.append(item[len(item)-1])
            extras_morphs.extend(list(set(temp)))
        except:
            pass
    elif decision == 'x':
        extras_morphs.append(lemma+'x')
    elif decision == 's':
        extras_morphs.append(lemma+'s')
    elif decision == 'iv':
        morph = str(input("which morph:"))
        

        temp = []
        test_verb = default_conjugator.conjugate(morph)
        all_conjugated_forms = test_verb.iterate()
        for item in all_conjugated_forms:
            if item not in temp:
                temp.append(item[len(item)-1])

        for variant in list(set(temp)):
            if variant is not None:
                new_word = word.replace(morph,variant)
                extras_morphs.append(new_word)

    return extras_morphs

In [193]:
"""
For eb_unk words: search through all articles, return 
sentences containing any words for unknown vocabulary. 

return dict object, key is lemma, then values are morphs found
and array of examples. 
"""

def find_unks(webpages,eb_unk_mo,prev_examples, prev_counter_examples):
    
    output = {}
    for webpage in webpages:
        try:

            request=urllib.request.Request(webpage,None,headers) #The assembled request
            response = urllib.request.urlopen(request)
            data = response.read()
            contents = text_from_html(data)

            contents_array = sent_tokenize(contents)

            for line in contents_array:
                tokenized = word_tokenize(line, language='french')
                for word in tokenized:
                    if word.lower() in eb_unk_mo:

                        if word.lower() in prev_examples:
                            if (line not in prev_examples[word.lower()]) and \
                            (line not in prev_counter_examples[word.lower()]):
                                if word.lower() not in output:
                                    output[eb_unk_mo[word.lower()]] = {} #key will be lemma
                                    output[eb_unk_mo[word.lower()]]['morph'] = []
                                    output[eb_unk_mo[word.lower()]]['sents'] = []

                                #disallow "À lire aussi" phrases
                                output[eb_unk_mo[word.lower()]]['morph'].append(word.lower())
                                output[eb_unk_mo[word.lower()]]['sents'].append(line)

        except:
            pass
    return output
    

In [194]:
def filter_text(webpage,text=None,print_word_lvl=False):


    #try:
    if text is None:
        request=urllib.request.Request(webpage,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read()
        contents = text_from_html(data)
    else:
        contents = webpage

    line_array = []
    percent_array = []
    contents_array = sent_tokenize(contents)

    disallowed_words = set()
    total_words = 0
    unknown_words = 0
    lines = []
    unknowns = []
    
    for line in contents_array:
        
        line_total = 0
        line_unks = 0
        tokenized = word_tokenize(line, language='french')

        unk_str = ""
        for word in tokenized:
            total_words +=1
            line_total +=1

            if not (bool(re.search("[0-9]", word)) or\
                    bool(re.search("[A-Z]", word)) or\
                    bool(re.search("[.,\/#!$%\^&\*;:{}=\-_`~()«»]", word)) or\
                    word.lower() in vocab['white_listed']):
                if not ((word.lower() in vocab['white_listed']) or (word.lower() in vocab['black_listed'])):
                    unk_str += '"'+word.lower()+'"'+", "

                disallowed_words.add(word.lower())
                unknown_words +=1
                line_unks +=1


        line_array.append(line)
        line_percent = (line_total-line_unks)/line_total
        percent_array.append(line_percent)

        if len(unk_str)>0:
            unk_str = unk_str[0:len(unk_str)-1]

        unknowns.append(unk_str)

    if print_word_lvl and total_words>0:
        print("word-level % known = "+str((1-(unknown_words/total_words))*100))

    return_pd = pd.DataFrame(list(zip(line_array,percent_array,unknowns)))
    return_pd.columns = ["line","line_percent","unk_words"]

    with open(path+'unknown_french_dad_list.txt',"w") as outfile:
        outfile.write(str(list(disallowed_words)))

    return return_pd
    
    #except:
    #    pass

##Dad section

In [195]:
path = "/Users/elyebliss/Desktop/Vocabulary/language_learning/vocab_dfs/"
source_file = "french_dad.json"

In [196]:
#read from existing .json formats
with open(path+source_file, "r") as path_in:
    vocab = json.loads(path_in.read())
vocab['white_listed'] = set(vocab['white_listed'])
vocab['black_listed'] = set(vocab['black_listed'])
print(len(vocab['white_listed']))

13599


In [197]:
            
##Journal en francais facile:
print("Checking Journal en francais facile")
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
#request=urllib.request.Request('https://savoirs.rfi.fr/en/apprendre-enseigner/langue-fran%C3%A7aise/journal-en-francais-facile',None,headers)
request=urllib.request.Request('https://francaisfacile.rfi.fr/fr/podcasts/journal-en-fran%c3%a7ais-facile/',None,headers)

resp = urllib.request.urlopen(request)
soup = bs4.BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))


pages = []
for link in soup.find_all('a', href=True):
    if '/fr/podcasts/le-journal-en-fran%C3%A7ais-facile/' in str(link['href']):
        #print(str(link['href']))
        str_page = 'https://savoirs.rfi.fr/'+str(link['href'])
        #str_page = 'https://savoirs.rfi.fr/en/apprendre-enseigner/langue-francaise/journal-en-francais-facile-'+str(re.search(r"[0-9]{8}.*",str(link['href'])).group(0))
        if str_page not in pages:
            pages.append(str_page)

for page in pages:
    print(page)

Checking Journal en francais facile
https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221201-sida-la-lutte-continue-massacre-en-ha%C3%AFti-l-op%C3%A9ra-de-paris-recrute-en-guyane
https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221130-rencontre-biden-macron-mort-d-un-ex-pr%C3%A9sident-chinois-la-baguette-fran%C3%A7aise-%C3%A0-l-unesco
https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221129-iran-300-morts-dit-le-r%C3%A9gime-emmanuel-macron-%C3%A0-washington-twitter-en-guerre-contre-apple
https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221128-manifestations-en-chine-hausse-du-covid-en-france-la-mobilisation-des-artistes-iran
https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221125-foot-le-s%C3%A9n%C3%A9gal-se-relance-poutine-parle-aux-mamans-de-soldats-pourparlers-au-venezuela
https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221124-prix-du-gaz-en-

In [200]:
with open(path+"text_input_web_dad_fr.txt","r") as infile:
    textfile = infile.read()
filtered_art = filter_text(textfile,text= True)
#display(filtered_art)

In [201]:
all_unks = set()

with open(path+'unknown_french_dad_list.txt',"r") as input_file:
    new_words = input_file.read()
    new_words = re.sub('\n',"",new_words)
    new_words = new_words.split(',')
    new_words = [re.sub("^['|\"]|^\['|['|\"]$|'\]$|\[","",line.strip().lower()) for line in new_words] #update regex
    for word in new_words:
        all_unks.add(word)
len(all_unks)

168

In [202]:
all_unks=list(all_unks)
unk_df = pd.DataFrame(all_unks)
unk_df.columns = ['word']
unk_df['status'] = pd.Series(['' for word in all_unks])

In [203]:
mitosheet.sheet(unk_df, analysis_to_replay="id-nzebdfviqb")

MitoWidget(analysis_data_json='{"analysisName": "id-nzebdfviqb", "analysisToReplay": null, "code": [], "stepSu…

In [204]:
add_ints = [3, 5, 8, 12, 13, 17, 20, 25, 27, 34, 47, 38, 40, 46, 52, 55, 57, 61, 66, 68, 73, 74, 78, 81, 82, 83, 84, 91, 96, 98, 100, 104, 106, 107, 112, 113, 115, 116, 117, 120, 121, 122, 124, 126, 132, 140, 141, 161,166]

In [205]:
for i in add_ints:
    # Set a cell value in status
    unk_df.at[i, 'status'] = 'k'

In [23]:
translations = []
for i in range(0,len(unk_df)):
    try:
        translations.append(ts.google(unk_df.word.iloc[i]))
    except:
        translations.append('')
unk_df['translations']=translations

In [24]:
mitosheet.sheet(unk_df, analysis_to_replay="id-ianxevwttg")

MitoWidget(analysis_data_json='{"analysisName": "id-ianxevwttg", "analysisToReplay": null, "code": [], "stepSu…

In [206]:
add_words = list(unk_df.word[unk_df['status']=='k'])
print(len(add_words))
for word in add_words:
    vocab['white_listed'].add(word)

49


In [207]:
filtered_art = filter_text(textfile,text= True)

In [208]:
filtered_art['theme']=['' for line in filtered_art.line]
filtered_art = filtered_art[['theme','unk_words','line','line_percent']]
with open(path+'dad_choose_themes.xlsx','wb') as outfile:
    filtered_art.to_excel(outfile, encoding='utf-8',index=False)

In [209]:
with open(path+'dad_choose_themes.xlsx','rb') as infile:
    filtered_art = pd.read_excel(infile)

In [210]:
themes = set(filtered_art.theme[filtered_art['theme']!='NaN'])
output = pd.DataFrame(columns=['theme','line'])
for theme in themes:
    temp = filtered_art[['theme','line']][filtered_art['theme']==theme]
    output = pd.concat([output,temp])
with open(path+'dad_article.xlsx','wb') as outfile:
    output.to_excel(outfile, encoding='utf-8',index=False)

In [211]:
#write to .json formats
df = vocab
df['white_listed'] = list(df['white_listed'])
df['black_listed'] = list(df['black_listed'])
with open(path+source_file, "w") as outfile:
    json.dump(df,outfile)
print(len(vocab['white_listed']))

13648


In [212]:
#read from existing .json formats
with open(path+source_file, "r") as path_in:
    vocab = json.loads(path_in.read())
vocab['white_listed'] = set(vocab['white_listed'])
vocab['black_listed'] = set(vocab['black_listed'])
print(len(vocab['white_listed']))

13648


##Filter for lessons

In [186]:
with open(path+"text_input_dad_fr.txt","r") as infile:
    textfile = infile.read()
filtered_art = filter_text(textfile,text= True)
display(filtered_art)

,line,line_percent,unk_words
0,Thoric a compris.,1.000000,
1,–J’ai compris.,1.000000,
2,"–Maintenant, il faut quitter la plage.",1.000000,
3,"Peu après, tous les Vikings ont pris leurs armes.",1.000000,
4,Ils ont traversé la forêt proche de la plage.,1.000000,
...,...,...,...
48,"Après avoir parlé avec le chef du village, le chef Eskol a réuni ses hommes.",0.941176,"""réuni"","
49,Il leur a dit :\n–J’ai appris beaucoup de choses.,1.000000,
50,Il y avait des Vikings ici avant nous.,1.000000,
51,Ils n’ont pas écouté les indigènes.,1.000000,


In [185]:
path

'/Users/elyebliss/Desktop/Vocabulary/language_learning/vocab_dfs/'

In [169]:
with open(path+'unknown_french_dad_list.txt',"r") as input_file:
    new_words = input_file.read()
    new_words = re.sub('\n',"",new_words)
    new_words = new_words.split(',')
    new_words = [re.sub("^['|\"]|^\['|['|\"]$|'\]$","",line.strip().lower()) for line in new_words] #update regex
#new_words
print(len(new_words))
known_manual = get_known(new_words)

13
lors
Known =k
12 remaining
vaste
Known =kk
got 1
11 remaining
indigènes
Known =kk
got 2
10 remaining
colline
Known =k
9 remaining
aliments
Known =k
8 remaining
réuni
Known =k
7 remaining
souvenirs
Known =k
6 remaining
cou
Known =k
5 remaining
rassuré
Known =k
4 remaining
poitrine
Known =k
3 remaining
seins
Known =k
2 remaining
genoux
Known =k
1 remaining
tatouages
Known =k
0 remaining


In [170]:
print(len(known_manual))
for word in known_manual:
    vocab['white_listed'].add(word)

2


In [182]:
#known_lines_only = list(set(list(filtered_art.knowns[filtered_art.knowns != "..."])))
#known_lines_only = sorted(known_lines_only,key=len)
filtered_art = filter_text(textfile,text= True)
known_lines_only = list(filtered_art.line[filtered_art['line_percent']==1.0])
known_lines_only = sorted(known_lines_only,key=len)


In [183]:
known_lines_only

['–a-t-il dit.',
 '–J’ai compris.',
 'Thoric a compris.',
 'Il était étrange.',
 'Et ils sont morts.',
 'Le chef Eskol a bu.',
 'Il faisait très chaud.',
 'Quelle scène étrange !',
 'Ce lieu était très vaste.',
 'Tout le groupe s’est arrêté.',
 'Il connaissait leur langue !',
 'Le reste du groupe l’a suivi.',
 'Certains sont morts de maladie.',
 'Les maisons semblaient étranges.',
 'Ils avaient essayé de les aider.',
 'Niels marchait en tête du groupe.',
 'Niels a fait un geste de la main.',
 '–On ne veut pas vous faire du mal !',
 'Le chef du village a regardé Eskol.',
 'Les Vikings avaient parlé avec lui.',
 'Ils n’ont pas écouté les indigènes.',
 '–Maintenant, il faut quitter la plage.',
 'Ils parlaient une langue très bizarre.',
 'Certains ont couru vers leurs maisons.',
 'Il lui a offert quelque chose à boire.',
 'Il y avait des Vikings ici avant nous.',
 'Les hommes ne savaient pas quoi penser.',
 'Puis il a dit « eau » en langue viking.',
 'Au début, les indigènes ont eu très pe

In [176]:
#export knowns: [filtered_art.knowns != "..."]
#known_list = list(filtered_art.knowns)
with open(path+"known_lines_dad_fr.txt","w") as outfile:
    for line in known_lines_only:
        outfile.write(line+"\n")

In [173]:
#write to .json formats
df = vocab
df['white_listed'] = list(df['white_listed'])
df['black_listed'] = list(df['black_listed'])
with open(path+source_file, "w") as outfile:
    json.dump(df,outfile)
print(len(vocab['white_listed']))

13599


In [174]:
#read from existing .json formats
with open(path+source_file, "r") as path_in:
    vocab = json.loads(path_in.read())
vocab['white_listed'] = set(vocab['white_listed'])
vocab['black_listed'] = set(vocab['black_listed'])
print(len(vocab['white_listed']))

13599


##EB section

In [213]:
#read eb unknown files from .json formats
"""
Remaining vocab size = 2218
Total learned = 149
"""
with open(path+'eb_unk.json', "r") as infile:
    eb_unk = json.loads(infile.read())
print("Remaining vocab size = "+str(len(eb_unk)))

with open(path+'eb_unk_mo.json', "r") as infile:
    eb_unk_mo = json.loads(infile.read())

with open(path+'eb_learned.json', "r") as infile:
    eb_learned= json.loads(infile.read())
print("Total learned = "+str(len(eb_learned)))

with open(path+'eb_unk_examples.json', "r") as infile:
    eb_unk_examples= json.loads(infile.read())

with open(path+'eb_unk_counter_examples.json', "r") as infile:
    eb_unk_counter_examples= json.loads(infile.read())

Remaining vocab size = 2212
Total learned = 155


In [128]:
"""
space = plain, add without conjurgating or changing
c = conjugate as normal verb
x = noun that adds x when plural
s = noun that add s when plural
iv = inner verb. There is a verb in the expression that should be 
    conjurgated
"""
with open(path+"fr_eb_extra.txt","r") as infile:
    extras_lemmas = infile.read().split('\n')

for lemma in extras_lemmas:
    extras_morphs = how_to_add(lemma)
    
    eb_unk[lemma] = extras_morphs
    for morph in extras_morphs:        
        if morph not in eb_unk_mo:
            eb_unk_mo[morph] = lemma

print("Remaining vocab size = "+str(len(eb_unk)))

#need to add above to examples and counter examples
for lemma in eb_unk:
    if lemma not in eb_unk_examples:
        eb_unk_examples[lemma] = []
    if lemma not in eb_unk_counter_examples:
        eb_unk_counter_examples[lemma]= []


c
x
s
iv
Remaining vocab size = 2237


Get larger list of pages to search

In [214]:
websites = set()
with open(path+'viewed_websites_fr.txt',"r") as infile:
    for line in infile.read().split('\n'):
        websites.add(line)
print(len(websites))

546


In [215]:
pages = []

In [216]:
##France24
print("Checking France24")
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
#request=urllib.request.Request('https://savoirs.rfi.fr/en/apprendre-enseigner/langue-fran%C3%A7aise/journal-en-francais-facile',None,headers)
request=urllib.request.Request('https://www.france24.com/fr/',None,headers)

resp = urllib.request.urlopen(request)
soup = bs4.BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))



for link in soup.find_all('a', href=True):
    str_ver = str(link['href'])
    
    #get regex such that links start with /fr/ and don't end
    #with /
    if bool(re.search('^\/fr\/.*',str_ver)) and (str_ver[len(str_ver)-1]!='/'):
        pages.append('https://www.france24.com'+str_ver)

Checking France24


In [217]:
##20minutes
print("Checking 20minutes")
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
#request=urllib.request.Request('https://savoirs.rfi.fr/en/apprendre-enseigner/langue-fran%C3%A7aise/journal-en-francais-facile',None,headers)
request=urllib.request.Request('https://www.20minutes.fr/',None,headers)

resp = urllib.request.urlopen(request)
soup = bs4.BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

for link in soup.find_all('a', href=True):
    str_ver = str(link['href'])
    
    final_segment = str_ver.split('/')[len(str_ver.split('/'))-1]
    #articles have https:// at the beginning and a date in the last segment
    if ('https://www.20minutes.fr' in str_ver)\
    and bool(re.search('[0-9]',final_segment)):
        pages.append(str_ver)

Checking 20minutes


In [218]:
##TV5Monde
print("Checking TV5Monde")
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
#request=urllib.request.Request('https://savoirs.rfi.fr/en/apprendre-enseigner/langue-fran%C3%A7aise/journal-en-francais-facile',None,headers)
request=urllib.request.Request('https://www.tv5monde.com/',None,headers)

resp = urllib.request.urlopen(request)
soup = bs4.BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

for link in soup.find_all('a', href=True):
    if '/info/' in str(link['href']):
        #print(str(link['href']))
        #str_page = 'https://savoirs.rfi.fr/'+str(link['href'])
        #str_page = 'https://savoirs.rfi.fr/en/apprendre-enseigner/langue-francaise/journal-en-francais-facile-'+str(re.search(r"[0-9]{8}.*",str(link['href'])).group(0))
        if str(link['href']) not in pages:
            pages.append(str(link['href']))


Checking TV5Monde


In [219]:
pages = list(set(pages))

#add in set_difference line here
print("new = "+str(len(set(pages).difference(websites))))
pages = list(set(pages).difference(websites))

new = 89


In [220]:
out = find_unks(pages,eb_unk_mo,eb_unk_examples,eb_unk_counter_examples)

In [221]:
review_out(out,eb_learned,eb_unk,eb_unk_mo,eb_unk_examples,eb_unk_counter_examples)

112 total
1 out of 112
Lemma: hisser

Morph: ['hisser', 'hisser']
Previous examples: 
 - — Martin Meissner/AP/SIPA        Après Vincent Luis en 2019 et 2020, un autre athlète tricolore vient de se hisser sur le toit du monde en triathlon .
New examples: 
Le succès japonais cumulé à la trop faible victoire allemande a permis aux Nippons, premiers du groupe E avec six points, de se hisser une nouvelle fois en 8es de finale avec les Espagnols, qui ont conservé une différence de buts plus favorable par rapport à l’Allemagne (+6 contre 1), les deux sélections terminant à quatre points.

options : k,m,a,q,s,ca
En 2000 et en 2004, elle avait quitté l’Euro dès le premier tour, avant de se hisser en demi-finales du Mondial 2006 à domicile deux ans plus tard.

options : k,m,a,q,s,ca
2 out of 112
Lemma: boulet

Morph: ['boulet']
Previous examples: 
 - Mais Schmeichel se détend bien et prouve pour la 187e fois de la saison qu’il n’a rien à voir, en sélection, avec le boulet que se traîne l’OGC Nic

options : k,m,a,q,s,cc
29/11/22 |                             Grève  Médecins libéraux et biologistes appelés à la grève pour soigner leurs…         Plus  d’actu                                          Retourner en haut de la page        Annonces Légales   Annonces légales  Allier  Alpes Maritimes  Bas Rhin  Bouches du Rhône  Côtes d'Armor  Finistère  Haut Rhin  Hauts de Seine  Loire  Loire Atlantique  Loiret  Maine et Loire  Marne  Paris  Seine Maritime  Val d'Oise  Val de Marne  Var  Vaucluse  Yvelines      Services   Distribution  Horoscope  Horoscope chinois  Conjugaison  Recette de cuisine  Résultats des élections  Maison  Le Journal des Seniors  Bébés et Mamans  Infolocale  Sportune   Codes promo   Codes promo  Code promo Uber Eats  Code promo ASOS  Code promo Bouygues Telecom  Code promo Lastminute  Code promo Tommy Hilfiger     20 Minutes   Recrutement  La 21 e Minute  20 Minutes Média  Panel des lecteurs  20 Minutes Livres  Charte   Jeux   Jeux gratuits  Mots fléchés  Solitai

options : k,m,a,q,s,cc
Imprimer          À lire aussi            28/11/22 |                             couleur de pop  Lizzo explique pourquoi l’industrie musicale est raciste selon elle              07/11/22 |                             HISTORIQUE  Dr. Dre a intronisé son protégé Eminem au Rock & Roll Hall of Fame              01/11/22 |                             DECES  Le rappeur américain Takeoff, du groupe Migos, tué par balle à 28 ans         Plus  d’actu                                          Retourner en haut de la page        Annonces Légales   Annonces légales  Allier  Alpes Maritimes  Bas Rhin  Bouches du Rhône  Côtes d'Armor  Finistère  Haut Rhin  Hauts de Seine  Loire  Loire Atlantique  Loiret  Maine et Loire  Marne  Paris  Seine Maritime  Val d'Oise  Val de Marne  Var  Vaucluse  Yvelines      Services   Distribution  Horoscope  Horoscope chinois  Conjugaison  Recette de cuisine  Résultats des élections  Maison  Le Journal des Seniors  Bébés et Mamans  Infolocale  Spo

options : k,m,a,q,s,cc
14 out of 112
Lemma: assise

Morph: ['assise']
New examples: 
Un frisson parcourt l'assistance un peu plus tôt quand la légende brésilienne, assise au premier rang, pose sa voix délicate sur ses accords de guitare devant le Chœur et l'Orchestre Philharmonique de Radio France.

options : k,m,a,q,s,ca
15 out of 112
Lemma: brasser

Morph: ['brasser']
New examples: 
Mais ce canevas n'est qu'un prétexte pour brasser ses thèmes favoris, amour, transmission et métissage.

options : k,m,a,q,s,ca
16 out of 112
Lemma: pionnier

Morph: ['pionnier', 'pionnier']
New examples: 
— Pixabay        Sur la page d’accueil de Google ce jeudi, vous verrez apparaître un homme jouant à la console de jeux vidéo : il s’agit de Gerald Lawson, pionnier dans le domaine.

options : k,m,a,q,s,ca
Le #GoogleDoodle du jour met à l'honneur Jerry Lawson, pionnier du jeu vidéo.

options : k,m,a,q,s,ca
17 out of 112
Lemma: précoce

Morph: ['précoce']
Previous examples: 
 - Ils s’ajoutent aux quelque 

options : k,m,a,q,s,ca
Ces accusations d’agression font écho à d’autres lancées par des spectatrices à l’occasion de l’avant-première d’un film, en mars dernier à Bruxelles, en Belgique.

options : k,m,a,q,s,ca
La scène de 7 minutes fait étrangement écho aux accusations d’agression sexuelle lancées par une jeune femme dans la vraie vie à l’égard de l’acteur et dramaturge français.

options : k,m,a,q,s,ca
27 out of 112
Lemma: dépens

Morph: ['dépens', 'dépens']
Previous examples: 
 - Le but de Christian Pulisic a suffi à envoyer les Américains en 8es de finale, aux dépens de cette surprenante sélection iranienne.
New examples: 
Joe Biden s'est engagé à ce que les créations d'emploi aux États-Unis ne se fassent pas "aux dépens de l'Europe", après des frictions autour de la politique industrielle de son pays.

options : k,m,a,q,s,ca
« Les Etats-Unis ne s’excuseront jamais pour le plan que nous allons mettre en place », a-t-il ajouté, tout en assurant que les créations d’emplois aux Etats-

...

options : k,m,a,q,s,ca
51 out of 112
Lemma: sacoche

Morph: ['sacoche']
New examples: 
Les plus : Sacoche et souris offerte ; Bon rapport performance/prix.

options : k,m,a,q,s,cc
52 out of 112
Lemma: fébrilement

Morph: ['fébrilement']
New examples: 
AP/Alessandra Tarantino       01 déc 2022  Mise à jour 01.12.2022 à 20:09 par TV5MONDE  avec AFP   Le Maroc, d'abord emballant contre le Canada (2-1) avant de finir fébrilement, a décroché pour la deuxième fois de son histoire un ticket en huitièmes de finale, prenant même la première place du groupe F du Mondial-2022 qu'il achève invaincu.

options : k,m,a,q,s,ca
53 out of 112
Lemma: dépit

Morph: ['dépit', 'dépit']
Previous examples: 
 - En dépit d’une relation là aussi étroite et historique, « le choix du Brexit et d’une politique unilatérale n’a pas été bien vu par les démocrates », décrypte pour 20 Minutes Karim Emile Bitar, directeur de recherches à l’Institut de relations internationales et stratégiques (Iris) et spécialiste de la 

options : k,m,a,q,s,ca
"Je ne regarde pas en arrière, je ne veux pas me focaliser sur le passé", assure André Ayew, le seul rescapé de cette rencontre au dénouement cruel pour son équipe.

options : k,m,a,q,s,ca
15h58 : C'est l'heure du dénouement dans ce passionnant groupe F !

options : k,m,a,q,s,ca
A 16 heures, dénouement du groupe F avec Croatie-Belgique et Canada-Maroc.

options : k,m,a,q,s,ca
Publicité         Lire la suite                 Premiers du groupe F grâce à leur victoire contre le Canada, les Marocains devaient regarder d'un œil attentif ce dénouement du groupe E pour connaître leur prochain adversaire en 8es de finale.

options : k,m,a,q,s,ca
Un dénouement improbable Le scénario est digne des plus grands manga.

options : k,m,a,q,s,cc
Un dénouement que peu auraient imaginé au tirage au sort, quand le duel entre l'Espagne et l'Allemagne était présenté comme le sommet des phases de poules.

options : k,m,a,q,s,ca
63 out of 112
Lemma: trappe

Morph: ['trappe', 'trappe', 

options : k,m,a,q,s,cs
72 out of 112
Lemma: rescapé

Morph: ['rescapé']
New examples: 
"Je ne regarde pas en arrière, je ne veux pas me focaliser sur le passé", assure André Ayew, le seul rescapé de cette rencontre au dénouement cruel pour son équipe.

options : k,m,a,q,s,ca
73 out of 112
Lemma: granit

Morph: ['granit']
New examples: 
A l'Euro en 2021, lors d'une précédente confrontation, les joueurs suisses Granit Xhaka et Xherdan Shaqiri avaient provoqué une immense colère en Serbie quand ils avaient mimé avec leurs mains l'aigle bicéphale albanais, un geste symbolique qui ne pouvait que réveiller les passions nationalistes dans les Balkans occidentaux.

options : k,m,a,q,s,cc
74 out of 112
Lemma: engouffrer

Morph: ['engouffrer']
New examples: 
Il interdit aussi aux concurrents de s’engouffrer dans la brèche.

options : k,m,a,q,s,ca
75 out of 112
Lemma: brèche

Morph: ['brèche', 'brèche']
New examples: 
Il interdit aussi aux concurrents de s’engouffrer dans la brèche.

options : k,

options : k,m,a,q,s,ca
92 out of 112
Lemma: maintes

Morph: ['maintes']
Previous examples: 
 - Le pays hôte a été maintes fois la cible de critiques notamment d'ONG internationales au sujet de l'exploitation des ouvriers sur les différents chantiers nécessaires à l'organisation du Mondial.
 - Sous une foule chantant à maintes reprises Vive le vent d’hiver , les Stentors ont enchaîné des chants de Noël sous un Grand sapin qui brillait de ses mille feux.
New examples: 
"Coordination" et "synchronisation", voilà deux mots répétés maintes fois par l'Élysée en amont et lors de cette visite d'État.

options : k,m,a,q,s,ck
93 out of 112
Lemma: malveillance

Morph: ['malveillance']
New examples: 
« Malveillance », « mensonge », « bêtise », lance le sénateur PS et président du groupe Paris en commun au Conseil de Paris.

options : k,m,a,q,s,ca
94 out of 112
Lemma: égout

Morph: ['égout']
New examples: 
Podcast « Animal » : Rat brun, rat d’égout, « Ratatouille », qui es-tu ?

options : k,m,a,q,s

options : k,m,a,q,s,ca
108 out of 112
Lemma: housse

Morph: ['housse']
Previous examples: 
 - Elle vient avec une housse idéale pour la protéger.
 - L’ ensemble Cyber Monday incluant 1 lit Emma Select, 1 matelas Emma Hybride, 1 à 2 oreillers Diamond Degree avec leurs taies blanches, 1 protège-matelas, 1 couette avec sa housse blanche, à 40% de réduction, soit à partir de 834,58€ au lieu de 1390,98€.
New examples: 
Planète Tarn : Dans cette entreprise ultramoderne, le recyclage des véhicules donne une seconde vie aux pièces auto Lille Nord : De la housse de siège au porte-carte grise… Ouikeep ou le bon sens du recyclage   Économie Voiture Automobile Recyclage Elisabeth Borne    Une faute ?

options : k,m,a,q,s,ca
109 out of 112
Lemma: consigner

Morph: ['consigne']
New examples: 
Imprimer          À lire aussi            30/11/22 |                             SECONDE VIE  Économie circulaire : Le recyclage de 100 % des métaux est-il un objectif…              22/09/22 |                  

In [ ]:
#could remove junk examples that include "À lire aussi"

In [222]:
for word in pages:
    websites.add(word)
with open(path+'viewed_websites_fr.txt',"w") as outfile:
    for line in websites:
        outfile.write(line+'\n')      

In [223]:
#write to eb unknown files to .json formats
"""
Remaining vocab size = 2200
Total learned = 167
"""
with open(path+'eb_unk.json', "w") as outfile:
    json.dump(eb_unk,outfile)
print("Remaining vocab size = "+str(len(eb_unk)))

with open(path+'eb_unk_mo.json', "w") as outfile:
    json.dump(eb_unk_mo,outfile)

with open(path+'eb_learned.json', "w") as outfile:
    json.dump(eb_learned,outfile)
print("Total learned = "+str(len(eb_learned)))

with open(path+'eb_unk_examples.json', "w") as outfile:
    json.dump(eb_unk_examples,outfile)

with open(path+'eb_unk_counter_examples.json', "w") as outfile:
    json.dump(eb_unk_counter_examples,outfile)

Remaining vocab size = 2200
Total learned = 167


### Code no longer in use

In [177]:
def return_line_percents(webpage):
    request=urllib.request.Request(webpage,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read()
    contents = text_from_html(data)

    known_array = []
    unk_array = []
    contents_array = sent_tokenize(contents)

    lines = []

    for line in contents_array:
        tokenized = word_tokenize(line, language='french')

        line_total = 0
        line_unks = 0
        for word in tokenized:

            line_total +=1

            if not (bool(re.search("[0-9]", word)) or\
                    bool(re.search("[A-Z]", word)) or\
                    bool(re.search("[.,\/#!$%\^&\*;:{}=\-_`~()«»]", word)) or\
                    word.lower() in vocab['white_listed']):

                    line_unks +=1

            lines.append(100*(line_total-line_unks)/line_total)
    return(lines)

In [ ]:

if (start is not None) and (stop is not None):
    contents_array=contents_array[max(start,0):min(stop,len(contents_array))]

with open(path+'unknown_french_dad_list.txt',"w") as outfile:
    outfile.write(str(list(disallowed_words)))


In [300]:
##add in extras
with open("/Users/elyebliss/Desktop/Vocabulary/vocab_dfs/fr_eb_extra.txt","r") as infile:
    extras_lemmas = infile.read().split('\n')

gs = goslate.Goslate()

default_conjugator = mlconjug3.Conjugator(language='fr')


for lemma in extras_lemmas:
    
    decision = how_to_add(lemma)
    
    if lemma not in eb_unk:
        eb_unk[lemma] = []
        
    extras_morphs = []
    extras_morphs.append(lemma)
    try:
        #if it's an infinitive, add all conjurgations
        test_verb = default_conjugator.conjugate(lemma)
        all_conjugated_forms = test_verb.iterate()
        for item in all_conjugated_forms:
            if item not in extras_morphs:
                extras_morphs.append(item[len(item)-1])
        extras_morphs = list(set(extras_morphs))
    except:
        
        #if not an expression:
        if len(lemma.split(' '))==1:
            #add plural nouns
            if lemma[len(lemma)-1]=='u':
                extras_morphs.append(lemma+'x')
            else:
                extras_morphs.append(lemma+'s')

    eb_unk[lemma] = extras_morphs
    for morph in extras_morphs:        
        if morph not in eb_unk_mo:
            eb_unk_mo[morph] = lemma

print("Remaining vocab size = "+str(len(eb_unk)))

Remaining vocab size = 2327


In [8]:
#Upload from download
with open(path+'known_french_dad_list.txt',"r") as input_file:
    new_words = input_file.read()
    new_words = re.sub('\n',"",new_words)
    new_words = new_words.split(',')
    new_words = [line.replace('"',"").strip().lower() for line in new_words]
    print("daily catch in word count:")
    print(len(new_words))
    
    for line in new_words:   
        vocab['white_listed'].add(line.replace('"',"").strip().lower())

FileNotFoundError: [Errno 2] No such file or directory: '/Users/elyebliss/Desktop/Vocabulary/vocab_dfs/known_french_dad_list.txt'

In [117]:
"""
to do:

"""

In [49]:
##INPUT-OUTPUT
#vocab list:
with open("/Users/elyebliss/Desktop/Vocabulary/vocab_dfs/dad_whitelisted.csv","r") as infile:
    whitelisted_lemmas = infile.read()


##VARIABLES
vocab_all = set()
gs = goslate.Goslate()

default_conjugator = mlconjug3.Conjugator(language='fr')


for line in whitelisted_lemmas.split('\n'):
    if len(line) > 0:
        #print(line)
        vocab = line.lower().strip()
        vocab_all.add(vocab)
        try:
            #if it's an infinitive, add all conjurgations
            test_verb = default_conjugator.conjugate(vocab)
            all_conjugated_forms = test_verb.iterate()
            for item in all_conjugated_forms:
                
                vocab_all.add(item[len(item)-1])
                
            
        except:
            #might be a noun, add plural
            vocab_all.add(add_noun(vocab))
#len(whitelisted_lemmas.split('\n'))            
#pp.pprint(vocab_all)

In [83]:
vocab = {}
vocab['white_listed'] = list(vocab_all)
vocab['black_listed'] = []


In [27]:
#used for creating eb data

with open(path+'eb_unk_lemmas.csv','r') as infile:
    eb_unk_lemmas = pd.read_csv(infile)
with open(path+'eb_unk_morph.csv','r') as infile:
    eb_unk_morph = pd.read_csv(infile)
    
eb_unk = {}
for word in eb_unk_lemmas.lemme:
    eb_unk[word] = []

    for index, row in eb_unk_morph.iterrows():
        if row.lemme == word:
            eb_unk[word].append(row.ortho)
eb_unk_mo = {}
for lemma in eb_unk.keys():
    for morph in eb_unk[lemma]:
        eb_unk_mo[morph] = lemma

In [147]:
eb_learned = {}
eb_unk_examples = {}
for word in eb_unk.keys():
    eb_unk_examples[word] = []

In [130]:
#restore to backups
eb_learned = eb_learned_backup
eb_unk = eb_unk_backup
eb_unk_mo = eb_unk_mo_backup
eb_unk_examples = eb_unk_examples